In [1]:
import pandas as pd
import numpy as np

In [2]:
train_inter = pd.read_csv('../dataset/raw/browsing_train.csv')
train_search = pd.read_csv('../dataset/raw/search_train.csv')

In [3]:
test_inter = pd.read_csv('../dataset/raw/browsing_test.csv')
test_search = pd.read_csv('../dataset/raw/search_test.csv')

In [4]:
sku = pd.read_csv('../dataset/raw/sku_to_content.csv')
sku.head()

,product_sku_hash,description_vector,category_hash,image_vector,price_bucket
0,26ce7b47f4c46e4087e83e54d2f7ddc7ea57862fed2e2a...,NaN,NaN,NaN,NaN
1,6383992be772b204a9ab75f86c86f5583d1bdd1222952d...,NaN,NaN,NaN,NaN
2,a2c3e2430c6ef9770b903ad08fa067a6b2b9db28f06e1b...,"[0.27629122138023376, -0.15763211250305176, 0....",06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,"[340.3592564184389, -220.19025864725685, 154.0...",7.0
3,1028ef615e425c328e7b95010dfb1fb93cf63749a1bc80...,"[0.4058118760585785, -0.03595402091741562, 0.2...",115a6a7017ee55752b8487c77dfde92b0d501d10a2e69c...,"[180.3463662921092, 222.702322343354, -8.88703...",8.0
4,9870c682d0d52d635501249da0eeaa118fad430b695ea1...,"[-0.3206155300140381, 0.01991105079650879, 0.0...",0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,"[-114.81079301576219, 84.55770104232334, 85.51...",2.0


In [5]:
all_inter = pd.concat([train_inter, test_inter], keys=['train', 'test'])
all_inter.head()

session_id_hash     event_type  \
train 0  20c458b802f6ea9374783bfc528b19421be977a6769785...  event_product   
      1  20c458b802f6ea9374783bfc528b19421be977a6769785...  event_product   
      2  20c458b802f6ea9374783bfc528b19421be977a6769785...       pageview   
      3  20c458b802f6ea9374783bfc528b19421be977a6769785...  event_product   
      4  20c458b802f6ea9374783bfc528b19421be977a6769785...       pageview   

        product_action                                   product_sku_hash  \
train 0         detail  d5157f8bc52965390fa21ad5842a8502bc3eb8b0930f3f...   
      1         detail  61ef3869355b78e11011f39fc7ac8f8dfb209b3442a9d5...   
      2            NaN                                                NaN   
      3         detail  d5157f8bc52965390fa21ad5842a8502bc3eb8b0930f3f...   
      4            NaN                                                NaN   

         server_timestamp_epoch_ms  \
train 0              1550885210881   
      1              1550885213307   
      2              1550885213307   
      3              1550885215484   
      4              1550885215484   

                                                hashed_url  
train 0  7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...  
      1  4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...  
      2  4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...  
      3  7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...  
      4  7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...

In [6]:
all_search = pd.concat([train_search, test_search], keys=['train', 'test'])
all_search.head()

session_id_hash  \
train 0  48fade624d47870058ce07dd789ccc04e46c70c0fa2a1b...   
      1  8731ca84ff7bb8cb647531d54e64feedb2519b4a7792a7...   
      2  9be980708345944960645d03606ea83b637cae9106b705...   
      3  9be980708345944960645d03606ea83b637cae9106b705...   
      4  9be980708345944960645d03606ea83b637cae9106b705...   

                                              query_vector clicked_skus_hash  \
train 0  [-0.20255649089813232, -0.016908567398786545, ...               NaN   
      1  [-0.007610442116856575, -0.14909175038337708, ...               NaN   
      2  [-0.20023074746131897, -0.03151938319206238, 0...               NaN   
      3  [-0.18556387722492218, -0.07620412111282349, 0...               NaN   
      4  [-0.03269264101982117, -0.27234694361686707, 0...               NaN   

                                         product_skus_hash  \
train 0                                                NaN   
      1  ['9ee9ffd7e2529a65f9a0b0c9eaae6330df85cf2e3af3...   
      2  ['7cc72dbed53bab78ec6a62feaa5052a7a1db7d201664...   
      3  ['62c4ddab6c1c81c74d315376b3c0dc7768c0286b3dc6...   
      4  ['2a0ee2924feabeec35e21e8fcb4d5b0684d190e46cef...   

         server_timestamp_epoch_ms  
train 0              1548575194779  
      1              1548276763869  
      2              1548937997295  
      3              1548938038268  
      4              1548938093827

In [7]:
def remap(df):
    _, map_ids = pd.factorize(df)
    return map_ids

# remap session ids

In [8]:
all_sess_ids = pd.concat([all_inter['session_id_hash'], all_search['session_id_hash']])
sess_map_ids = remap(all_sess_ids)
sess2idx = {n:i for i, n in enumerate(sess_map_ids)}
idx2sess = {i:n for i, n in enumerate(sess_map_ids)}
print('the number of sessions is {}'.format(len(sess2idx)))

the number of sessions is 5153009


# remap url ids

In [9]:
all_url_ids = all_inter['hashed_url']
url_map_ids = remap(all_url_ids)
url2idx = {n:i for i, n in enumerate(url_map_ids)}
idx2url = {i:n for i, n in enumerate(url_map_ids)}
print('the number of urls is {}'.format(len(url2idx)))

the number of urls is 498843


# remap item ids

In [10]:
sku_items = sku['product_sku_hash'].values

In [11]:
all_search['product_skus_hash'] = all_search['product_skus_hash'].apply(lambda x: eval(x) if not pd.isna(x) else [])
exp_items = np.concatenate(all_search['product_skus_hash'].values)

In [12]:
all_search['clicked_skus_hash'] = all_search['clicked_skus_hash'].apply(lambda x: eval(x) if not pd.isna(x) else [])
ckd_items = np.concatenate(all_search['clicked_skus_hash'].values)

In [21]:
all_item_ids = pd.DataFrame({'product_skus_hash': np.concatenate([sku_items, ckd_items, exp_items])}) 

In [22]:
item_map_ids = remap(all_item_ids['product_skus_hash'])

In [25]:
item2idx = {n:i for i, n in enumerate(item_map_ids)}
idx2item = {i:n for i, n in enumerate(item_map_ids)}
print('the number of items is {}'.format(len(idx2item)))

the number of items is 66386


# remap category ids

In [29]:
all_cate_ids = sku['category_hash']
cate_map_ids = remap(all_cate_ids)
cate2idx = {n:i for i, n in enumerate(cate_map_ids)}
idx2cate = {i:n for i, n in enumerate(cate_map_ids)}
print('the number of cates is {}'.format(len(cate2idx)))

the number of cates is 174


# remap action ids

In [46]:
all_inter['product_action'].fillna('pageview', inplace=True)
all_act_ids = all_inter['product_action']
act_map_ids = remap(all_act_ids)
act2idx = {n:i for i, n in enumerate(act_map_ids)}
idx2act = {i:n for i, n in enumerate(act_map_ids)}
print('the number of acts is {}'.format(len(act2idx)))

the number of acts is 5


In [51]:
act2idx

{'detail': 0, 'pageview': 1, 'add': 2, 'purchase': 3, 'remove': 4}

In [49]:
all_inter.drop(columns=['event_type'], inplace=True)

# map into csv

## sku

In [27]:
sku['product_sku_hash'] = sku['product_sku_hash'].map(item2idx)
sku['category_hash'] = sku['category_hash'].map(cate2idx)

In [52]:
sku.head()

,product_sku_hash,description_vector,category_hash,image_vector,price_bucket
0,0,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN
2,2,"[0.27629122138023376, -0.15763211250305176, 0....",0.0,"[340.3592564184389, -220.19025864725685, 154.0...",7.0
3,3,"[0.4058118760585785, -0.03595402091741562, 0.2...",1.0,"[180.3463662921092, 222.702322343354, -8.88703...",8.0
4,4,"[-0.3206155300140381, 0.01991105079650879, 0.0...",2.0,"[-114.81079301576219, 84.55770104232334, 85.51...",2.0


## inter

In [31]:
all_inter['product_sku_hash'] = all_inter['product_sku_hash'].map(item2idx)
all_inter['session_id_hash'] = all_inter['session_id_hash'].map(sess2idx)
all_inter['hashed_url'] = all_inter['hashed_url'].map(url2idx)
all_inter['product_action'] = all_inter['product_action'].map(act2idx)

In [53]:
all_inter.head()

session_id_hash  product_action  product_sku_hash  \
train 0                0               0           31231.0   
      1                0               0           30494.0   
      2                0               1               NaN   
      3                0               0           31231.0   
      4                0               1               NaN   

         server_timestamp_epoch_ms  hashed_url  
train 0              1550885210881           0  
      1              1550885213307           1  
      2              1550885213307           1  
      3              1550885215484           0  
      4              1550885215484           0

## search

In [55]:
all_search['session_id_hash'] = all_search['session_id_hash'].map(sess2idx)
all_search['clicked_skus_hash'] = all_search['clicked_skus_hash'].apply(lambda x: list(map(item2idx.get, x)))
all_search['product_skus_hash'] = all_search['product_skus_hash'].apply(lambda x: list(map(item2idx.get, x)))

In [61]:
all_search.head()

session_id_hash                                       query_vector  \
train 0          1498632  [-0.20255649089813232, -0.016908567398786545, ...   
      1          1498641  [-0.007610442116856575, -0.14909175038337708, ...   
      2          1498647  [-0.20023074746131897, -0.03151938319206238, 0...   
      3          1498647  [-0.18556387722492218, -0.07620412111282349, 0...   
      4          1498647  [-0.03269264101982117, -0.27234694361686707, 0...   

        clicked_skus_hash                                  product_skus_hash  \
train 0                []                                                 []   
      1                []                              [20133, 44952, 46326]   
      2                []   [24916, 65372, 1640, 55027, 40991, 52685, 36305]   
      3                []  [65035, 37475, 3766, 52978, 65850, 33711, 1248...   
      4                []  [22249, 44982, 16330, 32905, 7367, 12983, 7233...   

         server_timestamp_epoch_ms  
train 0              1548575194779  
      1              1548276763869  
      2              1548937997295  
      3              1548938038268  
      4              1548938093827

# save pkl

## eval str2list

In [62]:
all_search['query_vector'] = all_search['query_vector'].apply(eval)

In [65]:
sku['description_vector'] = sku['description_vector'].apply(lambda x: eval(x) if not pd.isna(x) else [])
sku['image_vector'] = sku['image_vector'].apply(lambda x: eval(x) if not pd.isna(x) else [])

## pkl

In [67]:
import pickle

In [69]:
with open('../dataset/new/sku_to_content.pkl', 'wb') as f:
    pickle.dump(sku, f)

In [70]:
with open('../dataset/new/search.pkl', 'wb') as f:
    pickle.dump(all_search, f)

In [71]:
with open('../dataset/new/browsing.pkl', 'wb') as f:
    pickle.dump(all_inter, f)

# info

In [72]:
map_info = {
    'item2idx': item2idx,
    'idx2item': idx2item,
    
    'act2idx': act2idx,
    'idx2act': idx2act,
    
    'cate2idx': cate2idx,
    'url2idx': url2idx,
    
    'idx2url': idx2url,
    'url2idx': url2idx,
    
    'sess2idx': sess2idx,
    'idx2sess': idx2sess
}

In [74]:
with open('../dataset/new/map_info.pkl', 'wb') as f:
    pickle.dump(map_info, f)